In [1]:
#Get Training Files
import os
import pandas as pd
in_data_array={}
out_data_array={}
for file in os.listdir('/kaggle/input/nfl-big-data-bowl-2026-prediction/train'):
    if file[0:2]=='in':
        week=int(file[-6:-4])
        dataFrame=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/train/'+file)
        in_data_array[week]=dataFrame
    elif file[0:2]=='ou':
        week=int(file[-6:-4])
        dataFrame=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/train/'+file)
        out_data_array[week]=dataFrame
print("done")

KeyboardInterrupt: 

In [4]:
import os
import pandas as pd
in_data_array={}
out_data_array={}
in_data_array[int('input_2023_w11.csv'[-6:-4])]=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/train/'+'input_2023_w11.csv')
out_data_array[int('output_2023_w11.csv'[-6:-4])]=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2026-prediction/train/'+'output_2023_w11.csv')
print(len(in_data_array[11]))
print(len(out_data_array[11]))

243413
27623


Will Seperate Training Data into Training and testing data to predict player position. For polynomial regression it is important to note that there are 8 gaurenteed frames per frame.

In [ ]:
from collections import namedtuple
#"Struct" to hold all frames belonging to player 
Player_Play_info= namedtuple('Player_Frame', ['Player_ID', 'Game_ID', 'Play_ID',"To_Track","Data_Frame"])
Play_info=namedtuple('Play_info',['Players','Play_id','to_track','out','frames',,'min_frame'])
# Sort the list by 'age' (ascending order, which is the default) the struct name in example is person
#sorted_by_age = sorted(people, key=lambda person: person.age)

In [ ]:
weeks=in_data_array.keys()
weeks=list(weeks)
weeks.sort()
plays_data={}
def turn_height_to_inches(oring):
    string=str(oring)
    arr=string.split('-')
    to_return=int(arr[0])*12+int(arr[1])
    return to_return
#df['motor type'].replace(['brushless', 'unspecified', 'brushed'],[0, 0.5, 1], inplace=True)
for i in weeks:
    print(i)
    in_data_array[i]['player_side'].replace(['Defense','Offense'],[-1,1],inplace=True)
    in_data_array[i]['play_direction'].replace(['left','right'],[-1,1],inplace=True)
    in_data_array[i]['player_height']=in_data_array[i]['player_height'].apply(turn_height_to_inches)
    #dirtydf['Date'] = pd.to_datetime(dirtydf['Date'], format = '%m/%d/%Y')
    in_data_array[i]['player_birth_date']=pd.to_datetime(in_data_array[i]['player_birth_date'],format='%Y-%m-%d')
    in_data_array[i].drop('player_name',axis=1,inplace=True)
    in_data_array[i]['player_role'].replace(['Defensive Coverage','Targeted Receiver','Passer','Other Route Runner'],[0,1,2,3],inplace=True)
    in_data_array[i]['player_position'].replace(['ILB', 'CB', 'OLB', 'MLB', 'FS', 'RB', 'WR', 'QB', 'TE', 'SS', 'DE', 'FB', 'DT', 'NT'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13],inplace=True)
print(in_data_array[i].info())
for i in weeks:
    for j in range(1,len(in_data_array[i])):
        row=in_data_array[i].head(j)
        play_id=str(row['game_id'][0])+"_"+str(row['play_id'][0])
        player_id=row['nfl_id']
        to_append=pd.DataFrame({'player_side':[row['play_direction'][0]],'absolute_yardline_number':[row['absolute_yardline_number'][0]],'nfl_id':[row['nfl_id'][0]],'frame_id':[row['frame_id'][0]],'player_height':[row['player_height'][0]],'player_weight':[row['player_weight'][0]],'player_birth_date':[row['player_birth_date'][0]],'player_position':[row['player_position'][0]],'player_side':[row['player_side'][0]],'player_role':[row['player_role'][0]],'x':[row['x'][0]],'y':[row['y'][0]],'s':[row['s'][0]],'a':[row['a'][0]],'dir':[row['dir'][0]],'o':[row['o'][0]]})
        num_frames=row['num_frames_output']
        if play_id in plays_data:
            plays_data[play_id].Data_Frame=pd.concat([plays_data[play_id].Data_Frame,to_append],axis=0)
            frame_num=row['frame_id'][0]
            if frame_num<plays_data[play_id].min_frame:
                plays_data[play_id].min_frame=frame_num
        else:
            #['Players','Play_id','to_track','out','frames','min_frame'])
            new_info=Play_info([player_id],play_id,-1,row['num_frames_output'],[to_append],row['frame_id'][0])            
            

11


/tmp/ipykernel_926/3429577402.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  in_data_array[i]['player_side'].replace(['Defense','Offense'],[-1,1],inplace=True)
/tmp/ipykernel_926/3429577402.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  in_data_array[i]['player_side'].replace(['Defense',

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243413 entries, 0 to 243412
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   game_id                   243413 non-null  int64         
 1   play_id                   243413 non-null  int64         
 2   player_to_predict         243413 non-null  bool          
 3   nfl_id                    243413 non-null  int64         
 4   frame_id                  243413 non-null  int64         
 5   play_direction            243413 non-null  int64         
 6   absolute_yardline_number  243413 non-null  int64         
 7   player_height             243413 non-null  int64         
 8   player_weight             243413 non-null  int64         
 9   player_birth_date         243413 non-null  datetime64[ns]
 10  player_position           243413 non-null  int64         
 11  player_side               243413 non-null  int64         
 12  pl

ValueError: If using all scalar values, you must pass an index

The minimum number of players is going to be 7 and the min number of fra